In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

import sys 
sys.path.append("../../") 
from QC5 import *
import numpy as np
from pyscf import gto,scf,fci,cc,mcscf,ao2mo,mp

In [2]:
n= 2
print('**********Hn=',n)
mol = gto.Mole()
mol.atom = genAn('H',n,0.74144)
mol.basis = 'sto3g' 
mol.charge = 0 
mol.spin = n%2
mol.symmetry = True
mol.build()   

# RHF
mf = scf.RHF(mol)
eRHF = mf.kernel()
print('eRHF',eRHF)

# classical information nelec and integrals
nelec = mol.nelec
norb = len(mf.get_occ())
ecore = mol.energy_nuc()
h1e = mf.get_hcore()
h2e = mf._eri  
mo_coeff = mf.mo_coeff 

# RCCSD amplitudes 
mycc = cc.CCSD(mf)
ccsd = mycc.kernel()
eCCSD = ccsd[0] + eRHF 
t1,t2  = ccsd[1],ccsd[2]

# FCI 
molFCI = fci.FCI(mf)
res = molFCI.kernel()
eFCI = res[0]
print('eFCI',res[0])

# RHF basis
fermion_transform = 'jordan_wigner'
h1,h2 = get_ao2so_h1_h2_int(mo_coeff,h1e,h2e)
qubit_hamiltonian = get_qubit_hamiltonian(ecore,h1,h2,fermion_transform)

**********Hn= 2
converged SCF energy = -1.11668219696281
eRHF -1.1166821969628056
E(CCSD) = -1.137269903217078  E_corr = -0.02058770625427266
eFCI -1.1372697371836944


In [9]:
#ucc_type = 'RuCCSD'
#cir,initial = get_ucc_ansatz(norb,nelec,ucc_type,ccsd_t1=t1,ccsd_t2=t2,fermion_transform='jordan_wigner')
#vqe = get_scipy_optimized_vqe(qubit_hamiltonian,cir,initial,printdetails=True)

In [5]:
cir

┏━━━┓ ┏━━━┓ ┏━━━━━━━━━┓                                     ┏━━━━━━━━━━┓ ┏━━━┓   
q0: ──┨ I ┠─┨╺╋╸┠─┨ RX(π/2) ┠───■─────────────────────────────■───┨ RX(7π/2) ┠─┨ H ┠─[red bold]↯[/]─
      ┗━━━┛ ┗━━━┛ ┗━━━━━━━━━┛   ┃                             ┃   ┗━━━━━━━━━━┛ ┗━━━┛   
      ┏━━━┓ ┏━━━┓             ┏━┻━┓                         ┏━┻━┓                      
q1: ──┨ I ┠─┨╺╋╸┠─────────────┨╺╋╸┠───■─────────────────■───┨╺╋╸┠────────────────────[red bold]↯[/]─
      ┗━━━┛ ┗━━━┛             ┗━━━┛   ┃                 ┃   ┗━━━┛                      
      ┏━━━┓ ┏━━━┓                   ┏━┻━┓ ┏━━━━━━━━━┓ ┏━┻━┓       ┏━━━┓ ┏━━━━━━━━━┓    
q2: ──┨ I ┠─┨ H ┠───────────────────┨╺╋╸┠─┨ RZ(-p0) ┠─┨╺╋╸┠───────┨ H ┠─┨ RX(π/2) ┠──[red bold]↯[/]─
      ┗━━━┛ ┗━━━┛                   ┗━━━┛ ┗━━━━━━━━━┛ ┗━━━┛       ┗━━━┛ ┗━━━━━━━━━┛    
      ┏━━━┓                                                                            
q3: ──┨ I ┠──────────────────────────────────────────────────────────────────────────[red bold]↯[/]─
      ┗━━━┛                                                                            
                                                                                       
                                         ┏━━━┓                                         
q0: ────■────────────────────────────■───┨ H ┠──────────────────────────────────────[red bold]↯[/]─ 
        ┃                            ┃   ┗━━━┛                                         
      ┏━┻━┓                        ┏━┻━┓ ┏━━━━━━━━━┓                                   
q1: ──┨╺╋╸┠───■────────────────■───┨╺╋╸┠─┨ RX(π/2) ┠────■───────────────────────────[red bold]↯[/]─ 
      ┗━━━┛   ┃                ┃   ┗━━━┛ ┗━━━━━━━━━┛    ┃                              
            ┏━┻━┓ ┏━━━━━━━━┓ ┏━┻━┓       ┏━━━━━━━━━━┓ ┏━┻━┓                            
q2: ────────┨╺╋╸┠─┨ RZ(p0) ┠─┨╺╋╸┠───────┨ RX(7π/2) ┠─┨╺╋╸┠───■─────────────────■───[red bold]↯[/]─ 
            ┗━━━┛ ┗━━━━━━━━┛ ┗━━━┛       ┗━━━━━━━━━━┛ ┗━━━┛   ┃                 ┃      
                                         ┏━━━┓              ┏━┻━┓ ┏━━━━━━━━━┓ ┏━┻━┓    
q3: ─────────────────────────────────────┨ H ┠──────────────┨╺╋╸┠─┨ RZ(-p0) ┠─┨╺╋╸┠─[red bold]↯[/]─ 
                                         ┗━━━┛              ┗━━━┛ ┗━━━━━━━━━┛ ┗━━━┛    
                                                                                       
                                                                  ┏━━━┓                
q0: ──────────────────────────────────────────────────────────────┨ H ┠──────────────[red bold]↯[/]─
                                                                  ┗━━━┛                
            ┏━━━━━━━━━━┓ ┏━━━┓                                    ┏━━━┓ ┏━━━┓          
q1: ────■───┨ RX(7π/2) ┠─┨ H ┠───■────────────────────────────■───┨ H ┠─┨ H ┠────────[red bold]↯[/]─
        ┃   ┗━━━━━━━━━━┛ ┗━━━┛   ┃                            ┃   ┗━━━┛ ┗━━━┛          
      ┏━┻━┓                    ┏━┻━┓                        ┏━┻━┓ ┏━━━━━━━━━┓          
q2: ──┨╺╋╸┠────────────────────┨╺╋╸┠───■────────────────■───┨╺╋╸┠─┨ RX(π/2) ┠────────[red bold]↯[/]─
      ┗━━━┛                    ┗━━━┛   ┃                ┃   ┗━━━┛ ┗━━━━━━━━━┛          
            ┏━━━┓ ┏━━━━━━━━━┓        ┏━┻━┓ ┏━━━━━━━━┓ ┏━┻━┓       ┏━━━━━━━━━━┓ ┏━━━┓   
q3: ────────┨ H ┠─┨ RX(π/2) ┠────────┨╺╋╸┠─┨ RZ(p0) ┠─┨╺╋╸┠───────┨ RX(7π/2) ┠─┨ H ┠─[red bold]↯[/]─
            ┗━━━┛ ┗━━━━━━━━━┛        ┗━━━┛ ┗━━━━━━━━┛ ┗━━━┛       ┗━━━━━━━━━━┛ ┗━━━┛   
                                                                                       
                                                         ┏━━━┓ ┏━━━━━━━━━┓             
q0: ────■────────────────────────────────────────────■───┨ H ┠─┨ RX(π/2) ┠────────[red bold]↯[/]─   
        ┃                                            ┃   ┗━━━┛ ┗━━━━━━━━━┛             
      ┏━┻━┓                                        ┏━┻━┓ ┏━━━┓ ┏━━━━━━━━━┓             
q1: ──┨╺╋╸┠───■────────────────────────────────■───┨╺

In [7]:
qubit_hamiltonian.compress()

-0.0989 [] +
 0.1712 [Z0] +
 0.1712 [Z1] +
-0.2228 [Z2] +
-0.2228 [Z3] +
 0.1686 [Z0 Z1] +
 0.0453 [X0 Y1 Y2 X3] +
-0.0453 [Y0 Y1 X2 X3] +
-0.0453 [X0 X1 Y2 Y3] +
 0.0453 [Y0 X1 X2 Y3] +
 0.1205 [Z0 Z2] +
 0.1659 [Z0 Z3] +
 0.1659 [Z1 Z2] +
 0.1205 [Z1 Z3] +
 0.1743 [Z2 Z3]

In [4]:
UCCSD = RuCCSD(nelec, t1, t2, fermion_transform)
UCCSD.parameter = 1.0 # set generalized amps=0

cir = UCCSD.params_circuit
initial = UCCSD.get_ccsd_initial_amps() # from pyscf CCSD
vqe = get_scipy_optimized_vqe(qubit_hamiltonian,cir,initial,printdetails=True)

sparse hamiltonian: True
VQE Runing under the BFGS optimizer with threshold 1e-06 maxniter default
Time: Thu Aug 22 11:39:56 2024; Iteration 0 expectation = -1.1372693562980427
Time: Thu Aug 22 11:39:56 2024; Iteration 1 expectation = -1.1372697371836995
Optimization terminated successfully.
         Current function value: -1.137270
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
-----------------  VQE Information Summary  -----------------
Success: True
Optimization terminated successfully.
Energy: -1.1372697371836995 
Amplitudes number: 2 
Time Consuming: 0.004 s          
Iteration Number: 1 
Max_gnorm: 3.3916110163436986e-10
Initial Amplitudes: 
[0.0, -0.11356162657045316]
Optimized Amplitudes: 
[2.125413211210706e-16, -0.11307629262527712]
--------------------------------------------------------------


In [8]:
UCCSD = RuCCGSD2(nelec, t1, t2, fermion_transform)
UCCSD.parameter = 0.1 # set generalized amps=0

cir = UCCSD.params_circuit
initial = UCCSD.get_ccsd_initial_amps() # from pyscf CCSD
vqe = get_scipy_optimized_vqe(qubit_hamiltonian,cir,initial,printdetails=True)

sparse hamiltonian: True
VQE Runing under the BFGS optimizer with threshold 1e-06 maxniter default
Time: Sun Apr 28 20:22:49 2024; Iteration 0 expectation = -1.1360509708135125
Time: Sun Apr 28 20:22:49 2024; Iteration 1 expectation = -1.137269292286313
Time: Sun Apr 28 20:22:49 2024; Iteration 2 expectation = -1.1372694388802405
Time: Sun Apr 28 20:22:49 2024; Iteration 3 expectation = -1.1372696543287915
Time: Sun Apr 28 20:22:49 2024; Iteration 4 expectation = -1.1372697351820298
Time: Sun Apr 28 20:22:49 2024; Iteration 5 expectation = -1.1372697371563578
Time: Sun Apr 28 20:22:49 2024; Iteration 6 expectation = -1.1372697371836695
Optimization terminated successfully.
         Current function value: -1.137270
         Iterations: 6
         Function evaluations: 8
         Gradient evaluations: 8
-----------------  VQE Information Summary  -----------------
Success: True
Optimization terminated successfully.
Energy: -1.1372697371836695 
Amplitudes number: 4 
Time Consuming: 0.017